# Program 1 — Keyword Router (Baseline)
This notebook evaluates a keyword-based routing baseline using the FinGuard-SDG 1160-question benchmark.


✅ Program 1 — Keyword Router: Training & Evaluation

This is not trained, but evaluated as a baseline.

In [ ]:
# 📌 Step 0 — Environment Setup (Mount Drive + Imports + Seed)

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
import json
from pathlib import Path
from sklearn.metrics import classification_report, accuracy_score
import warnings

warnings.filterwarnings("ignore")

SEED = 42
np.random.seed(SEED)

BASE_DIR = Path("/content/drive/MyDrive/FinGuardSDG")
DATA_DIR = BASE_DIR / "data"
RESULTS_DIR = BASE_DIR / "results" / "keyword"
MODELS_DIR = BASE_DIR / "models" / "keyword"

RESULTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print("Using BASE_DIR:", BASE_DIR)


Mounted at /content/drive
Using BASE_DIR: /content/drive/MyDrive/FinGuardSDG


In [ ]:
# 📌 Step 1 — Load Test Split from Google Drive

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from pathlib import Path

BASE_DIR = Path("/content/drive/MyDrive/FinGuardSDG")
SPLIT_DIR = BASE_DIR / "data" / "splits"

TEST_PATH = SPLIT_DIR / "FinGuard_SDG_test.csv"

print("Loading test set from:", TEST_PATH)

test_df = pd.read_csv(TEST_PATH)

print("Test shape:", test_df.shape)
test_df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading test set from: /content/drive/MyDrive/FinGuardSDG/data/splits/FinGuard_SDG_test.csv
Test shape: (174, 7)


,id,category,subcategory,question_text,answer_text,difficulty,source
0,Q-TVM-054,quantitative,time_value_of_money,"An investment of ₹1,50,000 earns 9% annually. ...","The future value is ₹2,73,832.14.",1,template
1,Q-EQ-047,quantitative,equity_valuation,A firm trades at a premium despite lower curre...,Investors expect future earnings growth.,2,literature-inspired
2,C-RR-011,conceptual,risk_return_theory,Why are risky assets expected to outperform ri...,Investors demand compensation for risk exposure.,1,literature-inspired
3,C-RR-020,conceptual,risk_return_theory,What limitation does variance have as a risk m...,It treats upside and downside deviations equally.,2,literature-inspired
4,Q-TVM-051,quantitative,time_value_of_money,"An annuity pays ₹48,000 annually for 9 years. ...","The present value is ₹2,87,184.93.",2,template


In [ ]:
# 📌 Step 2 — Define Keyword Map (Final version)

KEYWORD_MAP = {
    "quantitative": [
        "npv", "irr", "duration", "convexity", "calculate", "compute", "valuation",
        "coupon", "ytm", "yield", "volatility", "standard deviation", "beta",
        "sharpe", "sortino", "return", "portfolio", "annuity", "present value",
        "future value", "discount rate", "rate", "cash flow", "price"
    ],
    "conceptual": [
        "define", "explain", "compare", "difference", "what is", "why", "theory",
        "principle", "concept", "model", "framework", "interpret"
    ],
    "esg": [
        "esg", "environment", "social", "governance", "sustainability",
        "scope 1", "scope 2", "scope 3", "materiality", "transition risk",
        "carbon", "emissions", "disclosure", "sdg", "climate"
    ],
    "advisory": [
        "investor", "recommend", "advise", "should i", "should the investor",
        "allocate", "risk tolerance", "portfolio strategy",
        "given this scenario", "in this situation", "client"
    ]
}


In [ ]:
# 📌 Step 3 — Keyword Router

def keyword_router(query: str):
    query_l = query.lower()

    matched = {}
    for cat, kws in KEYWORD_MAP.items():
        hits = [kw for kw in kws if kw in query_l]
        if hits:
            matched[cat] = hits

    if not matched:
        return {"route": "unrouted", "confidence": 0.0, "matched_keywords": {}}

    # choose category with most matched keywords
    route = max(matched.keys(), key=lambda k: len(matched[k]))
    confidence = len(matched[route]) / max(len(KEYWORD_MAP[route]), 1)

    return {
        "route": route,
        "confidence": round(confidence, 2),
        "matched_keywords": matched
    }


In [ ]:
# 📌 Step 4 — Run Keyword Router on Test Set

rows = []
for _, row in test_df.iterrows():
    out = keyword_router(row["question_text"])
    rows.append({
        "id": row["id"],
        "question_text": row["question_text"],
        "true_category": row["category"],
        "predicted_category": out["route"],
        "confidence": out["confidence"],
        "matched_keywords": out["matched_keywords"],
        "router_used": "keyword"
    })

df_out = pd.DataFrame(rows)
df_out.head()


,id,question_text,true_category,predicted_category,confidence,matched_keywords,router_used
0,Q-TVM-054,"An investment of ₹1,50,000 earns 9% annually. ...",quantitative,unrouted,0.00,{},keyword
1,Q-EQ-047,A firm trades at a premium despite lower curre...,quantitative,unrouted,0.00,{},keyword
2,C-RR-011,Why are risky assets expected to outperform ri...,conceptual,conceptual,0.08,{'conceptual': ['why']},keyword
3,C-RR-020,What limitation does variance have as a risk m...,conceptual,unrouted,0.00,{},keyword
4,Q-TVM-051,"An annuity pays ₹48,000 annually for 9 years. ...",quantitative,quantitative,0.17,"{'quantitative': ['annuity', 'present value', ...",keyword


In [ ]:
# 📌 Step 5 — Evaluate Keyword Router Accuracy

y_true = df_out["true_category"]
y_pred = df_out["predicted_category"]

print("Keyword Router Accuracy:", accuracy_score(y_true, y_pred))

print("\nClassification Report:\n")

print(classification_report(
    y_true, y_pred,
    labels=["quantitative", "advisory", "conceptual", "esg"],
    zero_division=0
))


Keyword Router Accuracy: 0.46551724137931033

Classification Report:

              precision    recall  f1-score   support

quantitative       0.78      0.76      0.77        66
    advisory       1.00      0.10      0.19        39
  conceptual       0.31      0.53      0.39        36
         esg       0.73      0.24      0.36        33

   micro avg       0.57      0.47      0.51       174
   macro avg       0.70      0.41      0.43       174
weighted avg       0.72      0.47      0.48       174



In [ ]:
# 📌 Step 6 — Save Router Configuration

router_config = {
    "router_type": "keyword",
    "seed": SEED,
    "keyword_map": KEYWORD_MAP
}

config_path = MODELS_DIR / "keyword_router_config.json"

with open(config_path, "w") as f:
    json.dump(router_config, f, indent=2)

print("Saved keyword router config to:", config_path)


Saved keyword router config to: /content/drive/MyDrive/FinGuardSDG/models/keyword/keyword_router_config.json


In [ ]:
# 📌 Step 7 — Save Outputs

pred_path = RESULTS_DIR / "keyword_router_predictions.csv"
df_out.to_csv(pred_path, index=False)

summary = {
    "router": "keyword",
    "accuracy": float(accuracy_score(y_true, y_pred)),
    "classification_report": classification_report(
        y_true, y_pred,
        labels=["quantitative", "advisory", "conceptual", "esg"],
        zero_division=0,
        output_dict=True
    )
}

summary_path = RESULTS_DIR / "keyword_router_summary.json"

with open(summary_path, "w") as f:
    json.dump(summary, f, indent=2)

print("Saved keyword router results:")
print("-", pred_path)
print("-", summary_path)


Saved keyword router results:
- /content/drive/MyDrive/FinGuardSDG/results/keyword/keyword_router_predictions.csv
- /content/drive/MyDrive/FinGuardSDG/results/keyword/keyword_router_summary.json
